# Notebook to normalize the persistenc ein externals, using the darks as reference points


In [ ]:
from astropy.io import fits
import glob, os, shutil, pickle, bz2, gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import sigmaclip
from scipy.optimize import curve_fit
from scipy.special import gammaincc, gamma
from astropy.wcs import WCS
from astropy.stats import histogram
from itertools import product
from multiprocessing import Pool

%matplotlib notebook

In [ ]:
pwd

In [ ]:
#scsky = 2.75
#skyrad_i = 6
#skyrad_o = 15

#suff = '_sig'+'{:05.2f}'.format(scsky) + '_ri' + '{:05.2f}'.format(skyrad_i) + '_ro' + '{:05.2f}'.format(skyrad_o)

In [ ]:
# The project dir 
pdir = '/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence/'

#The mosaic dir
mdir = pdir+'/Mosaic_hi_res_folder/'

#The dir to save/load the Persistence curves dataframes
sdir = pdir+'/PD_dataframes_dir/'


In [ ]:
#Get all the exisitng computations for a given sky annulus (i.e. all existing sigma clipping leveles)

skyrad_i = 6
skyrad_o = 15

suff = '_sig*_ri' + '{:05.2f}'.format(skyrad_i) + '_ro' + '{:05.2f}'.format(skyrad_o)

filenames = sorted(glob.glob(sdir+'/Mean_res'+suff+'.pickle'))
for f in filenames:
    print(f)

In [ ]:
# Read all the results for different sigmas

mean_res_sigmas = []
for fn in filenames:
    with open(fn, 'rb') as f:
        mean_res_sigmas.append(pickle.load(f))


In [ ]:
# Read the file that contains the drak expsoure persistence measurements

with open(sdir+'/Mean_res.pickle', 'rb') as f:
    mean_res_drk = pickle.load(f)


In [ ]:
plt.style.use('bmh')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 13
plt.rcParams['axes.labelsize'] = 13
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['figure.titlesize'] = 13

ncols = 4
nrows =  (len(mean_res_drk['levels'])-1) // ncols

if ((len(mean_res_drk['levels'])-1) % ncols) > 0:
    nrows = nrows+1

fig, ax = plt.subplots(nrows, ncols, sharex=True, sharey=True, figsize=(5*ncols,3.5*nrows))


name_to_plot = ['ALL 25_PEXT']


for rr,mean_res in enumerate(mean_res_sigmas[5:]):

    ind_to_plot = [mean_res['Readme'].index(name) for name in name_to_plot]

    for i,(lev_dw,lev_up) in enumerate(zip(mean_res['levels'][0:-1],mean_res['levels'][1:])):

        k = i%ncols
        l = i//ncols

        for mm,j in enumerate(ind_to_plot):
            if 'PEXT' in name_to_plot[mm]:
                mrk = 'o'
                sz=12
            else:
                mrk = '*'
                sz=30

            if rr == 0:
                ax[l,k].plot(mean_res['med_times'][i][j],mean_res['med_pers'][i][j],
                         label='N: '+'{:3.1e}'.format(1.*np.sum(mean_res['Npoints'][i][j]))+' ; '+mean_res['Readme'][j],alpha=0.5)
            else:
                ax[l,k].plot(mean_res['med_times'][i][j],mean_res['med_pers'][i][j],alpha=0.5)




    
name_to_plot = []
for mrd in mean_res_drk['Readme']:
    
    t1 = 'ALL' in mrd
    t2 = '_PDRK' in mrd
    
    if t1&t2:
        name_to_plot.append(mrd)
    

ind_to_plot = [mean_res_drk['Readme'].index(name) for name in name_to_plot]
    
for i,(lev_dw,lev_up) in enumerate(zip(mean_res_drk['levels'][0:-1],mean_res_drk['levels'][1:])):
   
    k = i%ncols
    l = i//ncols
       
    for mm,j in enumerate(ind_to_plot):
        if 'PEXT' in name_to_plot[mm]:
            mrk = 'o'
            sz=12
        else:
            mrk = '*'
            sz=100
                        
        ax[l,k].scatter(mean_res_drk['med_times'][i][j],mean_res_drk['med_pers'][i][j],
                     label='N: '+'{:3.1e}'.format(1.*np.sum(mean_res_drk['Npoints'][i][j]))+' ; '+mean_res_drk['Readme'][j],s=sz, marker=mrk)
        
        
    
    ax[l,k].set_xscale('log')
    ax[l,k].set_yscale('log')
    meanlev = 0.5*(mean_res_drk['levels'][i]+mean_res_drk['levels'][i+1])
    titlestring = 'I:'+'{:d}, '.format(i)+'Log$_{10}$ (Stim.[e$^{-}$]): ' + '{:5.2f}'.format(meanlev) + '; ' + '{:6.2f}'.format(10**meanlev/80000.) +' x saturation' 
    ax[l,k].text(0.25,0.875,titlestring,verticalalignment='bottom', horizontalalignment='left',transform=ax[l,k].transAxes,bbox={'facecolor':'gray', 'alpha':.1, 'pad':1},fontsize=11)
    ax[l,k].legend(fontsize=9,loc=3)
    ax[l,k].set_ylim(0.009,15)
    ax[l,k].set_xlim(60,2000)
    ax[l,k].plot(np.array([1,10000]),700*np.array([1,1/10000.]),'--',c='black')

    if l==(nrows-1):
        ax[l,k].set_xlabel('Time [s]',fontsize=15)
    if k==0:
        ax[l,k].set_ylabel('Mean current [e$^{-}$/s]',fontsize=15)
    ax[l,k].set_axis_bgcolor('#FFFFFF')

    

plt.tight_layout(pad=.2)



In [ ]:
# A is the prediction at 1000 seconds

def intshpwl(t,A,index,t0=0.):
    tu = t[1:]
    td = t[:-1]

    if (index == -1.):
        return A*np.log( (tu+t0)/(td+t0) )
    else:
        return A/(1+index) * ( ((tu+t0)/1000)**(1+index) - ((td+t0)/1000)**(1+index) )#/(tu-td)


In [ ]:
name_to_fit = name_to_plot
ind_to_fit = ind_to_plot
intervals = [25.,100.,2.9,12.5,50]

for n,i,it in zip(name_to_fit,intervals,ind_to_fit):
    print(n,i,it)


In [ ]:
def Agamma_predictions(pars,ilev,thr=500.):

    residuals = []
    for j,i2f in enumerate(ind_to_fit):
        ct = mean_res_drk['med_times'][ilev][i2f]
        pers = mean_res_drk['med_pers'][ilev][i2f]
        
        semi_int = 0.5*intervals[j]
        for c,p in zip(ct,pers):
            if (c < thr):
                t = np.array([c-semi_int,c+semi_int])
                mod_pred = intshpwl(t,pars[0],pars[1])/(2*semi_int)
            
                #print(t,A,index,mod_pred)
                residuals.append(np.square(mod_pred-p))

    return np.sum(residuals)
    

        

In [ ]:
intshpwl(np.array([250-50,250+50]),500,-1.)/(100)

In [ ]:
from scipy.optimize import minimize

plt.style.use('bmh')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 13
plt.rcParams['axes.labelsize'] = 13
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['figure.titlesize'] = 13

In [ ]:
fig, ax = plt.subplots(len(mean_res_drk['levels'])-1,len(ind_to_fit), figsize=(3.75*len(ind_to_fit),3.5*(len(mean_res_drk['levels'])-1)))

for ilev in range(len(mean_res_drk['levels'])-1):

    meanlev = 0.5*(mean_res_drk['levels'][ilev]+mean_res_drk['levels'][ilev+1])
    

    for w,(thr,col) in enumerate(zip([500,750,1000,2000],['red','orange','magenta','yellow'])):
    
        bfres = minimize(Agamma_predictions, np.array([2000,-.87]), args=(ilev,thr), method='Powell')

        if(bfres['status'] == 0):
            print('Best fit results for level {:6.2f} and fit time up to {:7.1f}'.format(10**meanlev/80000.,thr))
            print(bfres['x'])
        else:
            print('Fit failed for level',meanlev,' fit time up to',thr)
            
            
        for j,i2f in enumerate(ind_to_fit):

            ct = mean_res_drk['med_times'][ilev][i2f]
            pers = mean_res_drk['med_pers'][ilev][i2f]
            semi_int = 0.5*intervals[j]
            mod_pred = np.empty_like(pers)
            
            for k,(c,p) in enumerate(zip(ct,pers)):
                t = np.array([c-semi_int,c+semi_int])
                mod_pred[k] = intshpwl(t,bfres['x'][0],bfres['x'][1])/(2*semi_int)
            
            ax[ilev,j].plot(ct,mod_pred,'--',c=col)
            ax[ilev,j].set_axis_bgcolor('#FFFFFF')
            
            if w == 3:
                ax[ilev,j].scatter(ct,pers,alpha=0.5,s=15)
                ax[ilev,j].set_xscale('log')
                ax[ilev,j].set_yscale('log')
                ax[ilev,j].set_title('{:5.1f}'.format(intervals[j])+', '+'{:6.2f}'.format(10**meanlev/80000.) +' x sat.',fontsize=15)
                ax[ilev,j].plot(np.array([1,10000]),700*np.array([1,1/10000.]),'--',c='black')
                ax[ilev,j].set_ylim(0.011,5)
                ax[ilev,j].set_xlim(85,5250)

    plt.tight_layout()